In [66]:
using Pkg
Pkg.activate("../.")
using Berry
versioninfo()
using PyPlot
using LinearAlgebra, SparseArrays
using StaticArrays
using BenchmarkTools
using Distributed
using SharedArrays
println("Actual number of threads used: ", Base.Threads.nthreads())

  Activating project at `~/Berry.jl`


Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 24 × Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, haswell)
  Threads: 25 on 24 virtual cores
Environment:
  JULIA_NUM_THREADS = auto
Actual number of threads used: 24


In [67]:
enum = enumerate
σˣ= @SMatrix [0  1; 1  0]
σᶻ= @SMatrix [1  0; 0 -1]
σʸ= @SMatrix [0 -im;im 0]
σ    = [σˣ, σʸ, σᶻ];

In [68]:
# Fukui et al. 2005
roll(n::Int,N::Int)  = ((n-1)%N)+1
function berry_curvature_2D(ψ::Array{ComplexF64,3})
    N,Nx,Ny = size(ψ)
    U_m = zeros(ComplexF64,Nx,Ny)
    U_n = zeros(ComplexF64,Nx,Ny)
    for i in 1:Nx
        for j in 1:Ny
            U_n[i,j] = ψ[:,i,j] ⋅ ψ[:,roll(i+1,Nx),j]
            U_n[i,j]/= abs(U_n[i,j])
            U_m[i,j] = ψ[:,i,j] ⋅ ψ[:,i,roll(j+1,Ny)]
            U_m[i,j]/= abs(U_m[i,j])
        end
    end
    Ω = zeros(Nx,Ny)
    for i in 1:Nx
        for j in 1:Ny
            Ω[i,j] = imag(log(U_n[i,j]*U_m[roll(i+1,Nx),j]*conj(U_n[i,roll(j+1,Ny)])*conj(U_m[i,j])))
        end
    end
    return Ω
end
# Distributed
function berry_curvature_2D(ψ::SharedArray{ComplexF64,3})
    println("Distributed function is used.")
    N,Nx,Ny = size(ψ)
    U_m = SharedArray{ComplexF64}(Nx,Ny)
    U_n = SharedArray{ComplexF64}(Nx,Ny)
    @distributed for i in 1:Nx
        for j in 1:Ny
            U_n[i,j] = ψ[:,i,j] ⋅ ψ[:,roll(i+1,Nx),j]
            U_n[i,j]/= abs(U_n[i,j])
            U_m[i,j] = ψ[:,i,j] ⋅ ψ[:,i,roll(j+1,Ny)]
            U_m[i,j]/= abs(U_m[i,j])
        end
    end
    Ω = SharedArray{Float64}(Nx,Ny)
    @distributed for i in 1:Nx
        for j in 1:Ny
            Ω[i,j] = imag(log(U_n[i,j]*U_m[roll(i+1,Nx),j]*conj(U_n[i,roll(j+1,Ny)])*conj(U_m[i,j])))
        end
    end
    return Ω
end

berry_curvature_2D (generic function with 2 methods)

# Spin models

In [1]:
function diagonalize_BHZ(Δ,Nkx=50,Nky=50)
    kx_all = LinRange(0,2π,Nkx)
    ky_all = LinRange(0,2π,Nky)
    H_BHZ(kx,ky,Δ) = sin(kx)*σˣ+ sin(ky)*σʸ+ (Δ + cos(kx)+cos(ky))*σᶻ
    E_all = SharedArray{Float64}(2,Nkx,Nky)
    ψ_all = SharedArray{ComplexF64}(2,2,Nkx,Nky)
    H = @SMatrix zeros(2,2)
    @distributed for ikx ∈ 1:Nkx
        kx = kx_all[ikx]
        for iky ∈ 1:Nky
            ky = ky_all[iky]
            H = H_BHZ(kx,ky,Δ)
            E_all[:,ikx,iky],ψ_all[:,:,ikx,iky] = eigen(H)
        end
    end
    return Array(E_all), Array(ψ_all)
end

LoadError: LoadError: UndefVarError: `@SMatrix` not defined
in expression starting at In[1]:7

In [2]:
E_all, ψ_all = diagonalize_BHZ(1.5,200,200);

LoadError: UndefVarError: `diagonalize_BHZ` not defined

In [3]:
Ω = berry_curvature_2D(ψ_all[:,1,:,:],distributed=true)
sum(Ω)/(2π)

LoadError: UndefVarError: `ψ_all` not defined